<a href="https://colab.research.google.com/github/Uiuran/engineering_exercises/blob/master/CommandQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Command and Query



In [0]:
#Structure to define specific query, a builder
class Builder:
  def __init__(self):
      ''' 
        Implement an abstract builder as chain of responsability

        For that each next_builder will be a instance inheriting Builder itself.
        add_builder function delegates the task of building to next builder by using add_builder inherited or
          by assigning to it if next_builder is None (i.e. the abstract class was called to create the inherited object but no builder was added yet).
        
        build() handles the task by calling the overriding method of the builder child classes through the recursive build classes of the parent.

      '''
      
      #self.person = Person()
      self.next_builder = None

  def build(self, *args, **kwargs):
      if self.next_builder:
          return self.next_builder.build(*args,**kwargs)

  def add_builder(self, builder):
      if self.next_builder:
        self.next_builder.add_builder(builder)
      else:
        self.next_builder = builder

# Job sender, sends any type of information, defined as query, query in this case can be an builder
class ImgPreprocessing:

  def __init__(self, *args,**kwargs):
      self.name = None
      self.img = None
      self.worker = None
      if "name" in kwargs:
        self.name = kwargs["name"]
      if "imgs" in kwargs:
        self.imgs = kwargs["imgs"]
      if "worker" in kwargs:
        self.worker = kwargs["worker"]   

      self.q = Query(name=self.name,imgs=self.imgs)
      self.worker.queries.append(self.q)  

  def apply(self):
      '''
       Instead of calling Builder itself, sends to client, event broker, to do the query.
      '''          
      self.worker.perform_query(self, self.q)                   
      return self

class StreamParams:

  def __init__(self, *args,**kwargs):
      self.name = None
      self.params = None
      self.worker = None
      if "name" in kwargs:
        self.name = kwargs["name"]
      if "params" in kwargs:
        self.params = kwargs["params"]
      if "worker" in kwargs:
        self.worker = kwargs["worker"]  

      self.q = Query(name=self.name,params=self.params)
      self.worker.queries.append(self.q)  

  def apply(self):
      '''
       Instead of calling Builder itself, sends to client, event broker, to do the query.
      '''    
      self.worker.perform_query(self, self.q)                   
      return self

#   Query defined anywhere else. A builder as example.
class Query(Builder):
    def __init__(self,*args, **kwargs):
        super().__init__()
        self.imgs = None
        self.params = None
        if "imgs" in kwargs:
            self.imgs = kwargs["imgs"]
        if "params" in kwargs:
            self.params = kwargs["params"]
        if "name" in kwargs:
            self.name = kwargs["name"]

    def __call__(self, *args, **kwargs):                
        if self.imgs:
            print(self.imgs)
        if self.params:
            print(self.params)
        super().build(*args,**kwargs)

# Event broker, worker
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:            
            item(*args, **kwargs)
        
# Query performer, receives information about sender and the query to be performed (usually named worker)
class Worker:
    def __init__(self):
        self.queries = Event()

    def perform_query(self,sender, query): 
        print(sender.name)
        self.queries(sender, query)

In [113]:
w = Worker()

# Senders receives worker object instantiated where the image will be processed
# since their query are builders chained together and allocated in worker w, when they send to worker the query is solved as a Chain Block
sender1 = ImgPreprocessing(name="img processor", worker=w, imgs = "images to be processed")
sender2 = StreamParams(name="stream requirer",worker=w, params = "some parametrization")

# senders sends imgs and information about itself to the worker, worker performs and print information about the sender.
sender2.apply()
print("\n")
sender1.apply()

stream requirer
images to be processed
some parametrization


img processor
images to be processed
some parametrization
